In [ ]:
#### APPENDIX 1 #####

In [ ]:
%matplotlib inline
%matplotlib widget
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import math
from scipy.optimize import minimize

In [ ]:
# Input Values

xySensor  = np.array([[0.1,-0.9,0.2,0.8,-0.6,0.3,0.5,-0.5,-0.01,-0.9],
                      [0.05,0.3,0.4,-0.3,0.3,-0.2,-0.84,0.85,-0.76,-0.9]])
#xSensor   = xySensor[0,:]
#ySensor   = xySensor[1,:]
outSensor = np.array([3.39382006,3.2073034,3.39965035,3.68810201,2.96941623,2.99495501,3.94274928,2.7968011,3.34929734,3.91296165])

discretization = 50
xPred =  np.linspace(-1, 1, discretization)
yPred =  np.linspace(-1, 1, discretization)
xPredMesh, yPredMesh = np.meshgrid(xPred,yPred)
xyPred = (np.array([xPredMesh.flatten(),yPredMesh.flatten()]))

MEAN = np.mean(outSensor)

noise = 1e-8
tau = 0.3
l = 0.2

In [ ]:
# Plot Data Points 

plt.figure(num='Data Points',figsize=(10,8))
fig1 = plt.axes(projection='3d')
fig1.scatter3D(xySensor[0,:], xySensor[1,:], outSensor, c='teal',label="Data Points",s=100)
locs, labels = plt.xticks()  # Get the current locations and labels.
plt.xticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.yticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
fig1.set_zlabel("Sensor Output")
plt.legend()
plt.show()

In [ ]:
# Function: Kernel 

def sqexp(x, xp, tauin, lin):
    """Squared exponential kernel (1 dimensional)
    
    Inputs
    ------
    x : (N), array of multiple inputs
    xp: float
    
    Returns
    -------
    cov (N,) -- Covariance between each input at *x* and the function values at *x*
    """
    cov = tauin**2 * np.exp(-1/2 * (x - xp)**2 / lin**2)
    return cov

def sqexp2d(x, xp, tauin, lin):
    """Squared exponential kernel (2 dimensional) """  
    xX   = x[0,:]
    xY   = x[1,:]
    xpX  = xp[0]
    xpY  = xp[1]
    tauX = tauin
    tauY = 1 # tauY=1 bc I only want one tau to tune
    sqexpX = sqexp(xX, xpX, tauX, lin)
    sqexpY = sqexp(xY, xpY, tauY, lin) 
    cov = sqexpX*sqexpY
    return cov

In [ ]:
# Function: Covariance 

def build_covariance(x, xp,tauin, lin, kern):
    """Build a covariance matrix
    
    Inputs
    -------
    x: (N) array of inputs
    xp: (M) array of inptus
    kern: a function mapping inputs to covariance
    
    Outputs
    -------
    cov: (N, M) covariance matrix
    """
    out = np.zeros((x.shape[1], xp.shape[1]))
    for jj in range(xp.shape[1]):
        out[:, jj] = kern(x, xp[:,jj], tauin, lin)
    return out

In [ ]:
# Function: GPR

def gpr(xtrain, ytrain, xpred, noise_var, mean_func, kernel):
    """Gaussian process regression Algorithm

    Inputs
    -------
    xtrain: (N, ) training inputs
    ytrain: (N, ) training outputs
    xpred:  (M, ) locations at which to make predictions
    noise_var: (N, ) noise at every training output
    mean_func: function to compute the prior mean
    kernel: covariance kernel
    
    Returns
    -------
    pred_mean : (M, ) predicted mean at prediction points
    pred_cov : (M, M) predicted covariance at the prediction points
    --
    """
    #xtrain  = np.transpose(xtrain)
    cov = build_covariance(xtrain, xtrain, tau, l, kernel)
    u, s, v = np.linalg.svd(cov)
    sqrtcov = np.dot(u, np.sqrt(np.diag(s)))

    noiseArray=(np.ones(np.size(xtrain[1,:])))*noise_var
    
    # pseudoinverse is better conditioned
    invcov = np.linalg.pinv(cov + np.diag(noiseArray))
    
    vec_pred = build_covariance(xpred, xtrain, tau, l, kernel)
    
    pred_mean = MEAN + np.dot(vec_pred, np.dot(invcov, ytrain - MEAN))
    cov_predict_pre = build_covariance(xpred, xpred, tau, l, kernel)
    cov_predict_up = np.dot(vec_pred, np.dot(invcov, vec_pred.T))
    pred_cov = cov_predict_pre - cov_predict_up
    #print(cov_predict_pre)
    
    return pred_mean, pred_cov

In [ ]:
# GPR Results

meanPred, covPred = gpr(xySensor, outSensor, xyPred, noise, MEAN, sqexp2d)

xPredMeshF, yPredMeshF = xPredMesh.flatten(),yPredMesh.flatten()
plt.figure(num='mean and cov',figsize=(10,8))
fig1 = plt.axes(projection='3d')
fig1.plot_trisurf(xPredMeshF,yPredMeshF,meanPred,cmap='winter',label='Predicted Mean')
fig1.plot_trisurf(xPredMeshF,yPredMeshF,np.diag(covPred),cmap='winter',label='Predicted Cov')
fig1.scatter3D(xySensor[0,:], xySensor[1,:], outSensor, c='k',label="Data Points", s=100)
locs, labels = plt.xticks()  # Get the current locations and labels.
plt.xticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.yticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
fig1.set_zlabel("Sensor Output")
plt.show()

In [ ]:
# GPR Results

meanPred, covPred = gpr(xySensor, outSensor, xyPred, noise, MEAN, sqexp2d)

xPredMeshF, yPredMeshF = xPredMesh.flatten(),yPredMesh.flatten()
plt.figure(num='mean and cov',figsize=(10,8))
fig1 = plt.axes(projection='3d')
fig1.scatter3D(xPredMeshF,yPredMeshF,meanPred,c='teal',label='Predicted Mean', alpha=0.2)
fig1.scatter3D(xPredMeshF,yPredMeshF,np.diag(covPred),c='palevioletred',label='Predicted Var', alpha=0.2)
fig1.scatter3D(xySensor[0,:], xySensor[1,:], outSensor, c='k',label="Data Points", s=100)
locs, labels = plt.xticks()  # Get the current locations and labels.
plt.xticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.yticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
fig1.set_zlabel("Sensor Output")
plt.legend()
plt.show()

In [ ]:
# Function: ± 2 Sigma Pred Computation

def twoSigma(inputMean, inputCov):
    varPred    = np.array(np.diag(inputCov))
    stdDevPred = np.sqrt(varPred)
    P2SigmaPred = inputMean + (2*stdDevPred)
    N2SigmaPred = inputMean - (2*stdDevPred)
    plt.figure(num='Pred Mean  ± 2 Sigma Pred Computation',figsize=(10,8))
    ax = plt.axes(projection='3d')
    ax.scatter3D(xPredMeshF,yPredMeshF,P2SigmaPred,c='limegreen',label='Predicted Mean + 2 Sigma', alpha=0.4, s=40)
    ax.scatter3D(xPredMeshF,yPredMeshF,meanPred,c='teal',label='Predicted Mean', alpha=0.4, s=40)
    ax.scatter3D(xPredMeshF,yPredMeshF,N2SigmaPred,c='maroon',label='Predicted Mean - 2 Sigma', alpha=0.4, s=40)
    locs, labels = plt.xticks()  # Get the current locations and labels.
    plt.xticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
    plt.yticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    ax.set_zlabel("Sensor Output")
    plt.legend()
    plt.show(ax)
    return(stdDevPred,P2SigmaPred,N2SigmaPred)

In [ ]:
stdDevMeanPred, P2SigmaMeanPred, N2SigmaMeanPred = twoSigma(meanPred, covPred)

In [ ]:
# Function: ± 2 Sigma Pred Computation

def twoSigma2(inputMean, inputCov):
    varPred    = np.array(np.diag(inputCov))
    stdDevPred = np.sqrt(varPred)
    P2SigmaPred = inputMean + (2*stdDevPred)
    N2SigmaPred = inputMean - (2*stdDevPred)
    plt.figure(num='Pred Mean  ± 2 Sigma Pred Computation',figsize=(10,8))
    ax = plt.axes(projection='3d')
    ax.plot_trisurf(xPredMeshF,yPredMeshF,meanPred,cmap='winter',label='Predicted Mean')
    ax.plot_trisurf(xPredMeshF,yPredMeshF,P2SigmaPred,cmap='plasma',label='Predicted Mean + 2 Sigma')
    ax.plot_trisurf(xPredMeshF,yPredMeshF,N2SigmaPred,cmap='plasma',label='Predicted Mean - 2 Sigma')
    locs, labels = plt.xticks()  # Get the current locations and labels.
    plt.xticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
    plt.yticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    ax.set_zlabel("Sensor Output")
    plt.show(ax)
    return(stdDevPred,P2SigmaPred,N2SigmaPred)



In [ ]:
# ± 2 Sigma Pred Computation
varPred,stdDevMeanPred, P2SigmaMeanPred, N2SigmaMeanPred = twoSigma2(meanPred, covPred)

In [ ]:
# STD Pred Plot
plt.figure(num='Pred Mean STD',figsize=(10,8))
fig3 = plt.axes(projection='3d')
fig3.plot_trisurf(xPredMeshF,yPredMeshF,meanPred,cmap='winter',label='Predicted Mean')
fig3.plot_trisurf(xPredMeshF,yPredMeshF,P2SigmaMeanPred,cmap='cool',label='Predicted Mean + 2 Sigma')
fig3.plot_trisurf(xPredMeshF,yPredMeshF,N2SigmaMeanPred,cmap='cool',label='Predicted Mean - 2 Sigma')
fig3.plot_trisurf(xPredMeshF, yPredMeshF, stdDevMeanPred,cmap='viridis',label="Predicted Points")
locs, labels = plt.xticks()  # Get the current locations and labels.
plt.xticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.yticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
fig3.set_zlabel("Sensor Output")
plt.show()

In [ ]:
# STD Pred Plot

plt.figure(num='Pred Mean STD',figsize=(10,8))
fig3 = plt.axes(projection='3d')
fig3.scatter3D(xPredMeshF,yPredMeshF,N2SigmaMeanPred,c='maroon',label='Predicted Mean - 2 Sigma', alpha=0.3)
fig3.scatter3D(xPredMeshF,yPredMeshF,meanPred,c='teal',label='Predicted Mean', alpha=0.3)
fig3.scatter3D(xPredMeshF,yPredMeshF,P2SigmaMeanPred,c='limegreen',label='Predicted Mean + 2 Sigma', alpha=0.3)
fig3.scatter3D(xPredMeshF, yPredMeshF, stdDevMeanPred,c='orange',label='Predicted Std Dev', alpha=0.3)
locs, labels = plt.xticks()  # Get the current locations and labels.
plt.xticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.yticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
fig3.set_zlabel("Sensor Output")
plt.legend()
plt.show()

In [ ]:
# Function: HyperParameters

def likelihood(x):
    tau_l, l_l, noise_l = x
    noiseArray=(np.ones(np.size(xySensor[0,:])))*noise_l
    cov = build_covariance(xySensor, xySensor, tau_l, l_l, sqexp2d)
    invcov = np.linalg.pinv(cov + np.diag(noiseArray))
    likelihoodOut = 0.5 * np.dot((outSensor.T-MEAN), np.dot(invcov, (outSensor-MEAN)))\
           + 0.5 * np.log(np.linalg.det(cov + np.diag(noiseArray)))\
           + 0.5 * np.size(xySensor[1,:]) * np.log(2*np.pi)
        
    return likelihoodOut

In [ ]:
# HyperParameters Results

noise = 1e-8
tau = 0.3
l = 0.2

xInit=np.array([tau, l, noise])
bounds = [(1e-15,None),(1e-15,None),(1e-15,None)]
minOut = minimize(likelihood, xInit, bounds=bounds, method='nelder-mead')
minOutX = np.array(minOut.x)
tau = minOutX[0]
l = minOutX[1]
noise = minOutX[2]
meanPred, covPred = gpr(xySensor, outSensor, xyPred, noise, MEAN, sqexp2d)
fig = plt.figure('Hyperparameter Pred Mean',figsize=(10,8))
fig4 = plt.axes(projection='3d')
fig4.plot_trisurf(xPredMeshF,yPredMeshF,meanPred,cmap='winter',label="Predicted Mean")
fig4.plot_trisurf(xPredMeshF,yPredMeshF,np.diag(covPred),cmap='winter',label="Predicted Cov")
fig4.scatter3D(xySensor[0,:], xySensor[1,:], outSensor, c='k',label="Data Points", s=100)
locs, labels = plt.xticks()  # Get the current locations and labels.
plt.xticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.yticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
fig4.set_zlabel("Sensor Output")
plt.show()

In [ ]:
plt.figure(num='Hyperparameter Pred Mean',figsize=(10,8))
fig4 = plt.axes(projection='3d')
fig4.scatter3D(xPredMeshF,yPredMeshF,meanPred,c='teal',label='Predicted Mean', alpha=0.4, s=40)
fig4.scatter3D(xPredMeshF,yPredMeshF,np.diag(covPred),c='palevioletred',label='Predicted Var', alpha=0.4, s=40)
fig4.scatter3D(xySensor[0,:], xySensor[1,:], outSensor, c='k',label="Data Points", s=100)
locs, labels = plt.xticks()  # Get the current locations and labels.
plt.xticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.yticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
fig3.set_zlabel("Sensor Output")
plt.legend()
plt.show()

In [ ]:
print(minOutX)

In [ ]:
# Hyperparameter ± 2 Sigma Pred Computation
stdDevMeanPred, P2SigmaMeanPred, N2SigmaMeanPred = twoSigma(meanPred, covPred)

In [ ]:
varPred = np.array(np.diag(covPred))

In [ ]:
# Hyperparameter STD Pred Plot

plt.figure(num='Pred Mean STD',figsize=(10,8))
fig5 = plt.axes(projection='3d')
#fig5.plot_trisurf(xPredMeshF,yPredMeshF,meanPred,cmap='winter',label='Predicted Mean')
#fig3.plot_trisurf(xPredMeshF,yPredMeshF,P2SigmaMeanPred,cmap='cool',label='Predicted Mean + 2 Sigma')
#fig3.plot_trisurf(xPredMeshF,yPredMeshF,N2SigmaMeanPred,cmap='cool',label='Predicted Mean - 2 Sigma')
fig5.plot_trisurf(xPredMeshF, yPredMeshF, stdDevMeanPred,cmap='viridis',label="Predicted Points")
#fig1.scatter3D(xySensor[0,:], xySensor[1,:], outSensor, c='k',label="Predicted Points", s=100)
locs, labels = plt.xticks()  # Get the current locations and labels.
plt.xticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.yticks(np.arange(-1, 1.25, step=0.25))  # Set label locations.
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
fig5.set_zlabel("Sensor Output")
plt.show()

In [ ]:
# Function : Finding indices of the peaks of a matrix

def findPeaks(inputMatrix):
    paddedMatrix = np.pad(inputMatrix,(1),'constant',constant_values=0)
    length1      = len(inputMatrix[0])
    length2      = len(inputMatrix[1])
    peakInd      = np.zeros((length1,length2))
    for xi in range(0,length1):
        for yi in range(0,length2):
            if paddedMatrix[xi+1,yi+1] > paddedMatrix[   xi , yi   ] and \
               paddedMatrix[xi+1,yi+1] > paddedMatrix[   xi , yi+1 ] and \
               paddedMatrix[xi+1,yi+1] > paddedMatrix[   xi , yi+2 ] and \
               paddedMatrix[xi+1,yi+1] > paddedMatrix[ xi+1 , yi   ] and \
               paddedMatrix[xi+1,yi+1] > paddedMatrix[ xi+1 , yi+2 ] and \
               paddedMatrix[xi+1,yi+1] > paddedMatrix[ xi+2 , yi   ] and \
               paddedMatrix[xi+1,yi+1] > paddedMatrix[ xi+2 , yi+1 ] and \
               paddedMatrix[xi+1,yi+1] > paddedMatrix[ xi+2 , yi+2 ]:
                peakInd[xi,yi] = 1
        
    return peakInd     

In [ ]:
# PredMatrix Peak Indices

meanPredMatrix       = meanPred.reshape(discretization,discretization)
meanPredPeakIndCoeff = findPeaks(meanPredMatrix)
meanPredPeak         = meanPredMatrix[meanPredPeakIndCoeff!=0]
print(meanPredPeak)

meanPredPeakIndX = xPredMesh * meanPredPeakIndCoeff
meanPredPeakIndX = meanPredPeakIndX[meanPredPeakIndX!=0]

meanPredPeakIndY = yPredMesh * meanPredPeakIndCoeff
meanPredPeakIndY = meanPredPeakIndY[meanPredPeakIndY!=0]

meanPredPeakInd  = np.array([meanPredPeakIndX,meanPredPeakIndY])
print(meanPredPeakInd)

fig6= plt.figure('PredMatrix Indices',figsize=(4,4))
plt.plot(meanPredPeakIndX,meanPredPeakIndY,'o',c='b')
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.show()

In [ ]:
# PredMatrix + 2 Sigma Peak Indices

meanPredP2Sigma = P2SigmaMeanPred
meanPredP2SigmaMatrix       = meanPredP2Sigma.reshape(discretization,discretization)
meanPredP2SigmaPeakIndCoeff = findPeaks(meanPredP2SigmaMatrix)
meanPredP2SigmaPeak         = meanPredP2SigmaMatrix[meanPredP2SigmaPeakIndCoeff!=0]
print(meanPredP2SigmaPeak)

meanPredP2SigmaPeakIndX = xPredMesh * meanPredP2SigmaPeakIndCoeff
meanPredP2SigmaPeakIndX = meanPredP2SigmaPeakIndX[meanPredP2SigmaPeakIndX!=0]

meanPredP2SigmaPeakIndY = yPredMesh * meanPredP2SigmaPeakIndCoeff
meanPredP2SigmaPeakIndY = meanPredP2SigmaPeakIndY[meanPredP2SigmaPeakIndY!=0]

meanPredP2SigmaPeakInd  = np.array([meanPredP2SigmaPeakIndX,meanPredP2SigmaPeakIndY])
print(meanPredP2SigmaPeakInd)

fig7= plt.figure('PredMatrix + 2 Sigma Peak Indices',figsize=(4,4))
plt.plot(meanPredP2SigmaPeakIndX,meanPredP2SigmaPeakIndY,'o',c='b')
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.show()

In [ ]:
# VarMatrix Peak Indices

#varPred = varPred
varPredMatrix       = varPred.reshape(discretization,discretization)
varPredPeakIndCoeff = findPeaks(varPredMatrix)
varPredPeak         = varPredMatrix[varPredPeakIndCoeff!=0]
print(varPredPeak)

varPredPeakIndX = xPredMesh * varPredPeakIndCoeff
varPredPeakIndX = varPredPeakIndX[varPredPeakIndX!=0]

varPredPeakIndY = yPredMesh * varPredPeakIndCoeff
varPredPeakIndY = varPredPeakIndY[varPredPeakIndY!=0]

varPredPeakInd  = np.array([varPredPeakIndX,varPredPeakIndY])
print(varPredPeakInd)


fig8= plt.figure('VarMatrix Peak Indices',figsize=(4,4))
plt.plot(varPredPeakIndX,varPredPeakIndY,'o',c='b')
plt.ylabel("Y Coordinate")
plt.xlabel("X Coordinate")

plt.show()